In [33]:
import librosa
import librosa.display
import numpy as np

def audio_to_melspectrogram(audio_path):
    y, sr = librosa.load(audio_path, sr=None, res_type="kaiser_fast")
    
    # Explicitly specifying arguments
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    log_melspectrogram = librosa.power_to_db(spectrogram).astype(np.float32)
    return log_melspectrogram



In [34]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Load the CSV files
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

# Group by filename to get all labels associated with a file
grouped_train_df = train_df.groupby('filename')['class'].apply(list).reset_index()

# One-hot encode the labels
mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform(grouped_train_df['class'])

# List of unique classes. You can find the num_classes from here.
num_classes = len(mlb.classes_)

# Extract audio file paths
train_audio_files = 'data/train/' + grouped_train_df['filename'].values
test_audio_files = 'data/test/' + test_df['filename'].values

# For the test set, we don't have the true labels. So we'll assume they're None for now.
test_labels = None

In [35]:
import torch.nn as nn
import torch.nn.functional as F

class BirdsongNet(nn.Module):
    def __init__(self, num_classes):
        super(BirdsongNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 512)  # Adjust this based on the size of your spectrograms
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 64 * 62)  # Adjust this based on the size of your spectrograms
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


In [36]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam

class BirdsongDataset(Dataset):
    def __init__(self, audio_files, labels=None):
        self.audio_files = audio_files
        self.labels = labels
        
    def __len__(self):
        return len(self.audio_files)
    
    def __getitem__(self, idx):
        audio_path = self.audio_files[idx]
        spectrogram = audio_to_melspectrogram(audio_path)
        
        # Zero-padding
        max_length = 500  # You can adjust this value based on your maximum expected spectrogram length.
        if spectrogram.shape[1] < max_length:
            padding = np.zeros((spectrogram.shape[0], max_length - spectrogram.shape[1]))
            spectrogram = np.hstack((spectrogram, padding))
        else:
            spectrogram = spectrogram[:, :max_length]
        
        if self.labels is not None:
            return torch.tensor(spectrogram).unsqueeze(0).float(), torch.tensor(self.labels[idx]).float()

        return torch.tensor(spectrogram).unsqueeze(0).float()



# Load your data into train and test loaders
train_dataset = BirdsongDataset(train_audio_files, train_labels)
train_loader = DataLoader(train_dataset, batch_size=31, shuffle=True)

test_dataset = BirdsongDataset(test_audio_files, test_labels)
test_loader = DataLoader(test_dataset, batch_size=31, shuffle=False)

# Initialize the model, criterion and optimizer
model = BirdsongNet(num_classes=num_classes).cuda()
criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.cuda(), labels.cuda()

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}")


RuntimeError: shape '[-1, 253952]' is invalid for input of size 1968128